# ch09. 装饰器 和 闭包

> 如果想掌握装饰器，那么必须理解闭包，即捕获函数主体外部定义的变量，也需要掌握 `nonlocal`

- Python 如何求解装饰器句法
- Python 如何判断变量是否为局部
- 闭包存在的原因和工作原理
- nonlocal 能解决什么问题
- 实现行为良好的装饰器
- 标准库中强大的装饰器：`@cache`, `@lru_cache`, `@singledispatch`;
- 实现一个参数化装饰器

## 装饰器的基础知识

装饰器是一种可调用对象，其参数是另一个函数（被装饰的函数）
装饰器可能会对被装饰的函数进行处理，然后返回函数，或者把函数替换为另一个函数/可调用对象

假如有一个名为 decorate 的装饰器

```python
@decorate
def target():
    print('running target()')
```

那么效果等同于下面的写法

```python
def target():
    print('running target()')

target = decorate(target)
```

<mark>装饰器通常会把一个函数替换为另一个函数</mark>

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

target()

running inner()


调用被装饰的 `target()`，运行的其实是 `inner`

In [2]:
target

<function __main__.deco.<locals>.inner()>

查看对象，target 现在是 inner 的引用

<mark>严格来说，装饰器只是语法糖。装饰器可以像常规的可调用对象那样调用，传入另一个函数</mark>

**<span style="color:green">装饰器有以下三个基本性质</span>**

- 装饰器是一个函数或其他可调用对象

- 装饰器可以把装饰的函数替换成别的函数

- 装饰器在加载模块时立即执行

## Python 何时执行装饰器

> 装饰器的一个关键的性质是：<mark>装饰器在被装饰的函数定义之后立即运行</mark>，通常是在导入时（比如加载模块时）

以下为一个示例 `registration.py`

In [3]:
registry = []       # 保存被 @register 装饰的函数的引用

def register(func):
    print(f'running register({func})')      # 为了演示，显示被装饰的函数
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')


def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()


running register(<function f1 at 0x107ef2a70>)
running register(<function f2 at 0x107ef2b90>)
running main()
registry -> [<function f1 at 0x107ef2a70>, <function f2 at 0x107ef2b90>]
running f1()
running f2()
running f3()


- `main()` 函数中首先显示 registry, 然后调用 f1(), f2(), f3()
- 只有把上面的示例作为**脚本运行**时，才能调用 `main()`

- 注意，register 在模块中其他函数之前运行了两次
- 在调用 register 时，传给它的参数是被装饰的函数

如果作为模块导入上面的文件，并不作为脚本运行，则输出如下的内容

```python
>>> import registration
running register(<function f1 at 0x107ef2a70>)
running register(<function f2 at 0x107ef2b90>)

```

<span style="color: green">上面的示例想强调的是，函数装饰器在导入模块时立即执行，而被装饰的函数只在显示调用时运行 </span>

## 注册装饰器

- 装饰器通常在一个模块中定义，然后再应用到其他模块中的函数上
- 大多数装饰器会在内部定义一个函数，然后将其返回

<mark>大多数装饰器会更改被装饰的函数。 通常的做法是，返回在装饰器内部定义的函数，取代被装饰的函数。</mark>

涉及内部函数的代码基本上离不开闭包，而理解闭包，需要熟悉Python中的变量作用域规则

## 变量作用域规则

看一个示例，关于全局变量和局部变量的

In [1]:
b = 6
def foo(a):
    print(a)
    print(b)
    b = 9

In [2]:
foo(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

如上所示，代码竟然报错了，报错信息为：`local variable 'b' referenced before assignment`
并不如预期的首先打印 b 的值：6，然后将 b 赋值 9，因为 b 是全局变量，而且局部变量在打印之后才赋值

<span style="color: green">Python 编译函数主体时，判断 b 是局部变量，因为在函数内给它赋值了。生成的字节码证实了这种判断</span>


所以，Python会尝试从局部变量中